<a href="https://colab.research.google.com/github/konkuk-gaegul/Fire-Safety-Prediction-AI-Contest/blob/main/AJH_Public_Data_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 활용 가능한 데이터 탐색
- 소상공인 데이터
- 어린이/노인/장애인 보호구역

In [1]:
# 경고 무시 명령
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pyproj
!pip install geopandas
!pip install furl

In [62]:
import pandas as pd
import numpy as np
from IPython.display import Image

## 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from pandas.plotting import scatter_matrix

## 지도 시각화를 위한 라이브러리
from folium import plugins
from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform
from shapely.geometry import Polygon
import geopandas as gpd

## 새로운 feature를 추가하기 위한 라이브러리
# 날짜
import time
import datetime
import holidays
# 날씨
import requests, bs4
from furl import furl
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [ ]:
def check_null(old_df):
    columns = old_df.columns
    for col in columns:
        bool_series =old_df[ old_df[col].isnull()]
        True_count = len( bool_series )
        print( col , '열의 결측치는' , True_count , '개 입니다')

## 전국 소상공인 데이터
- 2022년 06월
- 강원도 원주시 추출

In [57]:
df = pd.read_csv('/content/drive/MyDrive/GS리테일/data/소상공인시장진흥공단_상가정보/소상공인시장진흥공단_상가(상권)정보_강원_202206.csv')
df.head(5)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,25033300,동그라미중고타이어,NaN,D,소매,D23,자동차/자동차용품,D23A04,타이어판매,G45211,...,4215011100110960006010791,NaN,강원도 강릉시 가작로 270,210110,25488.0,1,NaN,NaN,128.904472,37.770252
1,17174549,세인트존스호텔Ohcrab,NaN,O,숙박,O01,호텔/콘도,O01A01,호텔/콘도,NaN,...,4215011300100010001017124,세인트존스호텔,강원도 강릉시 창해로 307,210120,25467.0,NaN,NaN,NaN,128.920908,37.791299
2,17174079,평창라마다호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,호텔/콘도,NaN,...,4276038024102450036000001,NaN,강원도 평창군 대관령면 오목길 107,232954,25342.0,NaN,NaN,NaN,128.717971,37.660051
3,17173904,호텔탑스텐스카이라운지,NaN,O,숙박,O01,호텔/콘도,O01A01,호텔/콘도,NaN,...,4215035029100920001000002,NaN,강원도 강릉시 옥계면 헌화로 455-34,210831,25633.0,NaN,NaN,NaN,129.052902,37.654680
4,24412526,레이디가구,NaN,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,...,4213011500111400020035715,NaN,강원도 원주시 송삼길 156-19,220150,26385.0,NaN,NaN,NaN,127.917307,37.327668


In [58]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [59]:
df_wonju = df[df.도로명주소.apply(lambda x : x.split(' ')[1] == '원주시')]
df_wonju.head(3)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
4,24412526,레이디가구,NaN,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,...,4213011500111400020035715,NaN,강원도 원주시 송삼길 156-19,220150,26385.0,NaN,NaN,NaN,127.917307,37.327668
6,17174157,치킨플러스원주기업도시점,원주기업도시점,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,4213033023114930000000001,원주기업도시라온프라이빗,강원도 원주시 지정면 가곡로 132,220821,26352.0,NaN,NaN,NaN,127.873974,37.383057
20,28515796,편백나무숲펜션,NaN,O,숙박,O03,캠프/별장/펜션,O03A01,캠프/별장/펜션,I55119,...,4213033022200530001000001,NaN,강원도 원주시 지정면 원양2로 25-14,220824,26359.0,NaN,1,NaN,127.797533,37.350421


In [60]:
df_wonju.to_csv('/content/drive/MyDrive/소방안전 AI  경진대회/원 데이터/sosang_wonju.csv' , index = False)

In [22]:
df_wonju.상권업종대분류명.value_counts().head(30)

음식          7420
소매          6980
생활서비스       3953
학문/교육       1402
부동산          714
관광/여가/오락     459
숙박           362
스포츠          194
Name: 상권업종대분류명, dtype: int64

In [23]:
df_wonju.상권업종중분류명.value_counts().head(30)

한식              3126
이/미용/건강         1489
종합소매점           1244
음/식료품소매          915
커피점/카페           913
가정/주방/인테리어       826
의복의류             803
유흥주점             633
부동산중개            619
자동차/이륜차          582
분식               580
닭/오리요리           509
세탁/가사서비스         506
일식/수산물           398
철물/난방/건설자재소매     310
패스트푸드            302
학원기타             285
자동차/자동차용품        280
사진/광학/정밀기기소매     275
학원-보습교습입시        266
화장품소매            263
개인/가정용품수리        245
제과제빵떡케익          243
학원-음악미술무용        243
양식               240
주택수리             239
건강/미용식품          232
중식               225
사무/문구/컴퓨터        222
학원-어학            205
Name: 상권업종중분류명, dtype: int64

In [21]:
df_wonju.상권업종소분류명.value_counts().head(30)

한식/백반/한정식        2093
여성미용실             954
커피전문점/카페/다방       872
부동산중개             619
편의점               459
자동차정비/카센타         396
갈비/삼겹살            357
후라이드/양념치킨         328
수퍼마켓              314
비만/피부관리           303
일반의류              295
호프/맥주             285
학원-입시             266
라면김밥분식            264
화장품판매점            257
청과물소매             256
유리/페인트/철물건축자재     242
세탁소/빨래방           227
국수/만두/칼국수         227
중국음식/중국집          224
청소/소독             212
식료품점              210
핸드폰소매             193
학원-외국어/어학         191
모텔/여관/여인숙         187
피아노/바이올린/기타       175
꽃집/꽃배달            173
제과점               162
일반가구소매            160
어린이집              160
Name: 상권업종소분류명, dtype: int64

In [25]:
df1 = pd.read_csv('/content/drive/MyDrive/소방안전 AI  경진대회/dataset.csv', encoding = 'euc-kr')
df1.head(2)

,GRID_ID,GRID_X_AXIS,GRID_Y_AXIS,OCRN_YMD,M00,M10,M15,M20,M25,M30,...,BCYC_ACDNT_OCRN_CNT,ELTRC_ACDNT_OCRN_CNT,POSNG_OCRN_CNT,ASPHYXIA_OCRN_CNT,FALLING_OCRN_CNT,FLAME_OCRN_CNT,CHMC_SBSTNC_ACDNT_OCRN_CNT,WETHR_ACDNT_OCRN_CNT,SXAL_ASALT_OCRN_CNT,BURN_OCRN_CNT
0,378508,378475,508475,2021-01-01,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,378509,378475,509475,2021-01-01,0.0,0.0,0.4,0.0,0.27,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df1.columns

Index(['GRID_ID', 'GRID_X_AXIS', 'GRID_Y_AXIS', 'OCRN_YMD', 'M00', 'M10',
       'M15', 'M20', 'M25', 'M30', 'M35', 'M40', 'M45', 'M50', 'M55', 'M60',
       'M65', 'M70', 'F00', 'F10', 'F15', 'F20', 'F25', 'F30', 'F35', 'F40',
       'F45', 'F50', 'F55', 'F60', 'F65', 'F70', 'DONG_CD', 'DONG_NM',
       'HGTPOJ_ACDNT_OCRN_CNT', 'PNTRINJ_OCRN_CNT', 'MCHN_ACDNT_OCRN_CNT',
       'ETC_OCRN_CNT', 'BLTRM_OCRN_CNT', 'ACDNT_INJ_OCRN_CNT',
       'EXCL_DISEASE_OCRN_CNT', 'VHC_ACDNT_OCRN_CNT', 'HRFAF_OCRN_CNT',
       'AGRCMCHN_ACDNT_OCRN_CNT', 'DRKNSTAT_OCRN_CNT',
       'ANML_INSCT_ACDNT_OCRN_CNT', 'FLPS_ACDNT_OCRN_CNT', 'UNKNWN_OCRN_CNT',
       'PDST_ACDNT_OCRN_CNT', 'LACRTWND_OCRN_CNT', 'MTRCYC_ACDNT_OCRN_CNT',
       'THML_DAMG_OCRN_CNT', 'DRV_ACDNT_OCRN_CNT', 'DRWNG_OCRN_CNT',
       'PRGNTW_ACDNT_OCRN_CNT', 'BCYC_ACDNT_OCRN_CNT', 'ELTRC_ACDNT_OCRN_CNT',
       'POSNG_OCRN_CNT', 'ASPHYXIA_OCRN_CNT', 'FALLING_OCRN_CNT',
       'FLAME_OCRN_CNT', 'CHMC_SBSTNC_ACDNT_OCRN_CNT', 'WETHR_ACDNT_

In [35]:
df1.DONG_CD.nunique()

74

In [36]:
df1.DONG_CD.unique()

array([4213035023, 4213035021, 4213035024, 4213025030, 4213025029,
       4213035025, 4213025024, 4213033022, 4213035026, 4213035022,
       4213033024, 4213025027, 4213033026, 4213025025, 4213025028,
       4213025021, 4213025026, 4213033021, 4213036025, 4213025023,
       4213025022, 4213036024, 4213036022, 4213036023, 4213033027,
       4213033023, 4213037024, 4213037023, 4213032030, 4213032029,
       4213036021, 4213033025, 4213032027, 4213032028, 4213037022,
       4213032025, 4213037021, 4213011500, 4213011800, 4213032021,
       4213032026, 4213032022, 4213038022, 4213011000, 4213011100,
       4213032024, 4213010900, 4213032023, 4213010700, 4213010600,
       4213010300, 4213011200, 4213031023, 4213038023, 4213011600,
       4213010500, 4213011300, 4213031022, 4213011700, 4213031024,
       4213031025, 4213031027, 4213038021, 4213011400, 4213031021,
       4213031026, 4213039022, 4213031028, 4213039023, 4213039025,
       4213039021, 4213039024, 4213039026, 4213039027])

In [34]:
df1.DONG_NM.nunique()

73

In [37]:
df1.DONG_NM.unique()

array(['정산리', '법천리', '흥호리', '대둔리', '반계리', '노림리', '취병리', '안창리', '단강리',
       '손곡리', '판대리', '후용리', '월송리', '궁촌리', '포진리', '문막리', '건등리', '간현리',
       '용암리', '비두리', '동화리', '주포리', '귀래리', '운계리', '보통리', '가곡리', '대안리',
       '사제리', '매호리', '산현리', '운남리', '신평리', '무장리', '용곡리', '매지리', '만종리',
       '흥업리', '무실동', '가현동', '주산리', '옥산리', '고산리', '서곡리', '단계동', '우산동',
       '광격리', '학성동', '대덕리', '단구동', '명륜동', '원동', '태장동', '장양리', '신촌리',
       '관설동', '개운동', '봉산동', '의관리', '반곡동', '흥양리', '수암리', '둔둔리', '금대리',
       '행구동', '평장리', '교항리', '금창리', '학곡리', '구학리', '성남리', '신림리', '황둔리',
       '송계리'], dtype=object)

## 어린이보호구역

In [116]:
kid_df = pd.read_csv('/content/drive/MyDrive/소방안전 AI  경진대회/원 데이터/전국어린이보호구역표준데이터.csv' , encoding = 'euc-kr')
kid_zone_wonju = kid_df[kid_df.제공기관명.str.contains('원주시')].reset_index()
kid_zone_wonju.drop(columns = ['index'] , inplace= True)
kid_zone_wonju.head(3)

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
0,유치원,한아름,강원도 원주시 송림길 78,강원도 원주시 단구동 1455-3,37.330458,127.955658,강원도 원주시청,원주경찰서,Y,1.0,10,2022-04-28,4190000,강원도 원주시
1,유치원,큰나무,강원도 원주시 북원로2704번길 29-1,강원도 원주시 태장동 1369-3,37.385125,127.950218,강원도 원주시청,원주경찰서,Y,1.0,10,2022-04-28,4190000,강원도 원주시
2,유치원,학림,강원도 원주시 판부면 서곡널다리길 18-1,강원도 원주시 판부면 서곡리 534-13,37.322068,127.936254,강원도 원주시청,원주경찰서,Y,1.0,10,2022-04-28,4190000,강원도 원주시


In [117]:
# 도로명 주소에 NaN값 하나 포착
kid_zone_wonju[kid_zone_wonju.소재지도로명주소.isnull()]

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
46,초등학교,샘마루초등학교,NaN,강원도 원주시 지정면 가곡리 1507-1,37.379499,127.871104,강원도 원주시청,원주경찰서,Y,2.0,30,2022-04-28,4190000,강원도 원주시


In [118]:
# 지번 주소에는 NaN값 없음
kid_zone_wonju[kid_zone_wonju.소재지지번주소.isnull()]

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명


In [121]:
kid_zone_wonju.iloc[ 46 : 47 , :].소재지도로명주소 = kid_zone_wonju.iloc[ 46 : 47 , :].소재지지번주소
kid_zone_wonju.iloc[ 46 : 47 , :]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
46,초등학교,샘마루초등학교,강원도 원주시 지정면 가곡리 1507-1,강원도 원주시 지정면 가곡리 1507-1,37.379499,127.871104,강원도 원주시청,원주경찰서,Y,2.0,30,2022-04-28,4190000,강원도 원주시


In [122]:
kid_zone_wonju.to_csv('/content/drive/MyDrive/소방안전 AI  경진대회/원 데이터/전국_어린이보호구역_원주시_추출.csv', index = False)

## 노인장애인보호구역

In [178]:
old_df = pd.read_csv('/content/drive/MyDrive/소방안전 AI  경진대회/원 데이터/전국노인장애인보호구역표준데이터.csv' , encoding = 'euc-kr')
old_df_wonju = old_df[old_df.시군구명 == '원주시']
old_df_wonju.head(3)

,장소유형코드,대상시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,제한속도,관리기관명,관리기관전화번호,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
1857,1.0,노인종합복지관,강원도,원주시,42130,강원도 원주시 동부순환로 9-6,강원도 원주시 단구동 1486-14,37.318658,127.969307,30,강원도 원주시,033-737-3512,원주경찰서,N,NaN,10~20,2022-03-03,4190000,강원도 원주시
1858,2.0,주.단기보호센터 꿈꾸는나무,강원도,원주시,42130,강원도 원주시 치악로1534번길 14-3,강원도 원주시 단구동 1473-4,37.321472,127.966528,30,강원도 원주시,033-737-3512,원주경찰서,Y,1.0,8,2022-03-03,4190000,강원도 원주시
1859,1.0,황둔송계경로당,강원도,원주시,42130,강원도 원주시 신림면 신림황둔로 1214,강원도 원주시 신림면 황둔리 333-3,37.261670,128.189420,30,강원도 원주시,033-737-3512,원주경찰서,N,NaN,10,2022-03-03,4190000,강원도 원주시


In [179]:
def check_null(old_df):
    columns = old_df.columns
    for col in columns:
        bool_series =old_df[ old_df[col].isnull()]
        True_count = len( bool_series )
        print( col , '열의 결측치는' , True_count , '개 입니다')

In [180]:
check_null(old_df_wonju)

장소유형코드 열의 결측치는 0 개 입니다
대상시설명 열의 결측치는 0 개 입니다
시도명 열의 결측치는 0 개 입니다
시군구명 열의 결측치는 0 개 입니다
시군구코드 열의 결측치는 0 개 입니다
소재지도로명주소 열의 결측치는 1 개 입니다
소재지지번주소 열의 결측치는 0 개 입니다
위도 열의 결측치는 0 개 입니다
경도 열의 결측치는 0 개 입니다
제한속도 열의 결측치는 0 개 입니다
관리기관명 열의 결측치는 0 개 입니다
관리기관전화번호 열의 결측치는 0 개 입니다
관할경찰서명 열의 결측치는 0 개 입니다
CCTV설치여부 열의 결측치는 0 개 입니다
CCTV설치대수 열의 결측치는 4 개 입니다
보호구역도로폭 열의 결측치는 0 개 입니다
데이터기준일자 열의 결측치는 0 개 입니다
제공기관코드 열의 결측치는 0 개 입니다
제공기관명 열의 결측치는 0 개 입니다


In [ ]:
# 위도 경도의 결측치는 삭제
# idx = old_df[['위도' , '경도']].dropna(axis = 0).index
# old_df_GPS_drop = old_df.loc[ idx , :]
# old_df_GPS_drop.head(2)

In [ ]:
# check_null(old_df_GPS_drop)

In [ ]:
# old_df_GPS_drop[ ['소재지도로명주소' , '소재지지번주소'] ].isnull()

In [182]:
# 주소 둘 다 NaN값인 애들은 없다
old_df_wonju[ (old_df_wonju.소재지도로명주소.isnull() == True) ]

,장소유형코드,대상시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,제한속도,관리기관명,관리기관전화번호,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
1861,1.0,원주보훈요양원,강원도,원주시,42130,NaN,강원도 원주시 반곡동 1956-1,37.330255,127.991283,30,강원도 원주시,033-737-3512,원주경찰서,Y,1.0,40,2022-03-03,4190000,강원도 원주시


In [186]:
old_df_wonju.loc[ 1861 : 1861 , :].소재지도로명주소 = old_df_wonju.loc[ 1861 : 1861 , :].소재지지번주소
old_df_wonju.loc[ 1861 : 1861 , :]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,장소유형코드,대상시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,제한속도,관리기관명,관리기관전화번호,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
1861,1.0,원주보훈요양원,강원도,원주시,42130,강원도 원주시 반곡동 1956-1,강원도 원주시 반곡동 1956-1,37.330255,127.991283,30,강원도 원주시,033-737-3512,원주경찰서,Y,1.0,40,2022-03-03,4190000,강원도 원주시


In [187]:
check_null(old_df_wonju)

장소유형코드 열의 결측치는 0 개 입니다
대상시설명 열의 결측치는 0 개 입니다
시도명 열의 결측치는 0 개 입니다
시군구명 열의 결측치는 0 개 입니다
시군구코드 열의 결측치는 0 개 입니다
소재지도로명주소 열의 결측치는 0 개 입니다
소재지지번주소 열의 결측치는 0 개 입니다
위도 열의 결측치는 0 개 입니다
경도 열의 결측치는 0 개 입니다
제한속도 열의 결측치는 0 개 입니다
관리기관명 열의 결측치는 0 개 입니다
관리기관전화번호 열의 결측치는 0 개 입니다
관할경찰서명 열의 결측치는 0 개 입니다
CCTV설치여부 열의 결측치는 0 개 입니다
CCTV설치대수 열의 결측치는 4 개 입니다
보호구역도로폭 열의 결측치는 0 개 입니다
데이터기준일자 열의 결측치는 0 개 입니다
제공기관코드 열의 결측치는 0 개 입니다
제공기관명 열의 결측치는 0 개 입니다


In [188]:
old_df_wonju.to_csv('/content/drive/MyDrive/소방안전 AI  경진대회/원 데이터/전국노인장애인보호구역_원주_추출.csv' , index = False)